In [4]:
# The following imports are useful in the numerai tournament
# Most of them are not being used in this file

# Numerai API
from numerapi import NumerAPI

# data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# stats
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_squared_error

# machine learning models
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# other
import gc
import json
from tqdm import trange
from itertools import product
import functools
import random
from timeit import default_timer
import re
import time
from pprint import pprint
from copy import deepcopy
from varname import nameof
from datetime import datetime

# save variables
import pickle
import joblib

In [5]:
with open('data/features.json', 'r') as f:
    FEATURE_METADATA = json.load(f)
del f

FEATURES_L = list(FEATURE_METADATA['feature_stats'].keys())
FEATURES_M = FEATURE_METADATA['feature_sets']['medium']
FEATURES_S = FEATURE_METADATA['feature_sets']['small']
FEATURES_2 = FEATURE_METADATA['feature_sets']['v2_equivalent_features']
FEATURES_3 = FEATURE_METADATA['feature_sets']['v3_equivalent_features']
FEATURES_N = FEATURE_METADATA['feature_sets']['fncv3_features']

ERA = 'era'
DATA = 'data_type'
Y_TRUE = 'target_nomi_v4_20'
Y_PRED = 'target_prediction'
Y_RANK = 'prediction' 

X_COLS = FEATURES_L
COLUMNS = [ERA, DATA] + X_COLS + [Y_TRUE]

df_feature_metadata = pd.DataFrame(FEATURE_METADATA['feature_stats'])
df_feature_metadata

,feature_honoured_observational_balaamite,feature_polaroid_vadose_quinze,feature_untidy_withdrawn_bargeman,feature_genuine_kyphotic_trehala,feature_unenthralled_sportful_schoolhouse,feature_divulsive_explanatory_ideologue,feature_ichthyotic_roofed_yeshiva,feature_waggly_outlandish_carbonisation,feature_floriated_amish_sprite,feature_iconoclastic_parietal_agonist,...,feature_circumspective_daughterly_brubeck,feature_mimetic_sprawly_flue,feature_inductile_umbrian_wallah,feature_ineloquent_bihari_brougham,feature_shakespearean_alpha_constituent,feature_marxian_plated_refrigeration,feature_amative_irresponsive_flattie,feature_intermissive_coronal_reinsertion,feature_dwarfish_isochronal_amateur,feature_polyphyletic_unplumed_pandiculation
legacy_uniqueness,0.177814,0.241351,0.659092,0.234994,0.471051,0.608926,0.220884,0.671897,0.878900,0.174533,...,0.777375,0.788337,0.798391,0.777608,0.781240,0.801397,0.812955,0.824060,0.793313,0.806686
spearman_corr_w_target_nomi_20_mean,-0.000796,0.000199,-0.000619,0.001724,0.000661,-0.001529,-0.000623,-0.003439,0.000012,-0.001762,...,-0.005185,-0.005866,-0.006759,-0.005132,-0.005950,-0.001996,-0.002635,-0.003977,-0.001651,-0.002001
spearman_corr_w_target_nomi_20_sharpe,-0.078689,0.020379,-0.067669,0.127591,0.065213,-0.173158,-0.084122,-0.323518,0.001100,-0.161949,...,-0.216388,-0.252259,-0.310896,-0.219533,-0.247247,-0.094700,-0.130897,-0.205489,-0.077834,-0.095218
spearman_corr_w_target_nomi_20_reversals,0.000074,0.000082,0.000067,0.000095,0.000072,0.000058,0.000052,0.000076,0.000105,0.000086,...,0.000385,0.000365,0.000324,0.000386,0.000393,0.000324,0.000306,0.000288,0.000342,0.000313
spearman_corr_w_target_nomi_20_autocorr,-0.013665,0.110503,0.036986,0.149465,0.014508,0.023341,-0.066927,0.093560,-0.041187,-0.127240,...,0.007061,0.014354,0.006205,-0.009986,0.013370,-0.011183,-0.002933,-0.007453,-0.020823,-0.012853
spearman_corr_w_target_nomi_20_arl,3.650350,3.456954,2.916201,3.984733,3.702128,3.411765,3.262500,3.575342,3.017341,3.984733,...,3.782609,3.434211,3.650350,3.625000,3.625000,3.866667,3.755396,3.434211,3.503356,3.503356


In [6]:
targets = list(pd.read_parquet('data/train.parquet'))

In [8]:
targets = [t for t in targets if t.startswith('target')]
targets = [t for t in targets if t.endswith('20')]

In [10]:
joblib.dump(targets, 'target_names.pkl')

['target_names.pkl']